In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Irritable_bowel_syndrome_(IBS)"
cohort = "GSE36701"

# Input paths
in_trait_dir = "../../input/GEO/Irritable_bowel_syndrome_(IBS)"
in_cohort_dir = "../../input/GEO/Irritable_bowel_syndrome_(IBS)/GSE36701"

# Output paths
out_data_file = "../../output/preprocess/Irritable_bowel_syndrome_(IBS)/GSE36701.csv"
out_gene_data_file = "../../output/preprocess/Irritable_bowel_syndrome_(IBS)/gene_data/GSE36701.csv"
out_clinical_data_file = "../../output/preprocess/Irritable_bowel_syndrome_(IBS)/clinical_data/GSE36701.csv"
json_path = "../../output/preprocess/Irritable_bowel_syndrome_(IBS)/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Gene expression analysis of rectal mucosa in chronic irritable bowel syndrome (IBS) compared to healthy volunteers (HV)"
!Series_summary	"An investigation of gene expression changes in rectal biopsies from donors with IBS compared to controls to begin to understand this complex syndrome. To further investigate differences between IBS groups (constipation and diarrhoea predominant) (part1) and how IBS relates to bacterial infection (part2) with biopsies taken 6 months after Campylobacter jejuni infection."
!Series_overall_design	"Part1: 18 Constipation predominant IBS subjects (IBS-C) and 27 diarrhoea predominant IBS subjects (IBS-D) compared to 21 healthy volunteers (HV)."
!Series_overall_design	"Part2: 21 Campylobacter jejuni infection (PIBD, PIBS, PINIBS) compared to 19 healthy volunteers (HV)."
!Series_overall_design	"PIBD = post Campylobacter infection with IBS (within 6 months)"
!Series_overall_design	"PIBS = post infection IBS (unknown time 

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import os
import pandas as pd
import numpy as np
import json
from typing import Dict, Any, Optional, List, Callable, Union

# 1. Gene Expression Data Availability
# Looking at the background info, this appears to be a gene expression study in rectal biopsies
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability

# For trait, we can see disease status at index 3 in the sample characteristics
trait_row = 3  # 'disease: HV', 'disease: IBS-D', 'disease: IBS-C', etc.

# Age is not available in the sample characteristics
age_row = None

# Gender is available at index 1
gender_row = 1  # 'gender: M', 'gender: F'

# 2.2 Data Type Conversion
def convert_trait(value: str) -> int:
    """Convert IBS disease status to binary (0 for healthy, 1 for IBS)"""
    if value is None or ":" not in value:
        return None
    
    disease = value.split(":", 1)[1].strip().upper()
    
    # Healthy volunteers
    if disease == "HV":
        return 0
    # IBS cases (IBS-D, IBS-C, PIBD, PIBS)
    elif disease.startswith("IBS") or disease == "PIBD" or disease == "PIBS":
        return 1
    # PINIBS = post infection with no IBS
    elif disease == "PINIBS":
        return 0
    else:
        return None

def convert_gender(value: str) -> int:
    """Convert gender to binary (0 for female, 1 for male)"""
    if value is None or ":" not in value:
        return None
    
    gender = value.split(":", 1)[1].strip().upper()
    
    if gender == "F":
        return 0
    elif gender == "M":
        return 1
    else:
        return None

# No convert_age function needed since age data is not available

# 3. Save Metadata
# Determine if trait data is available
is_trait_available = trait_row is not None
validate_and_save_cohort_info(is_final=False, cohort=cohort, info_path=json_path, 
                             is_gene_available=is_gene_available, 
                             is_trait_available=is_trait_available)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Assuming clinical_data is already loaded from a previous step
    # If not, we need to load it from the cohort directory
    clinical_data_path = os.path.join(in_cohort_dir, "clinical_data.csv")
    if os.path.exists(clinical_data_path):
        clinical_data = pd.read_csv(clinical_data_path)
    else:
        # Using the first available matrix file in the directory
        matrix_files = [f for f in os.listdir(in_cohort_dir) if f.endswith('_series_matrix.txt')]
        if matrix_files:
            matrix_path = os.path.join(in_cohort_dir, matrix_files[0])
            # Assuming there's a function to extract clinical data from matrix file
            # For now, we'll use a placeholder approach
            with open(matrix_path, 'r') as f:
                lines = f.readlines()
            
            # Find sample characteristic lines
            sample_chars = [line for line in lines if line.startswith('!Sample_characteristics_ch1')]
            
            # Create a simple dataframe with sample characteristics
            sample_ids = [line for line in lines if line.startswith('!Sample_geo_accession')]
            if sample_ids:
                sample_ids = [s.strip().split('\t')[1].replace('"', '') for s in sample_ids]
                
                # Transpose the data for proper format
                clinical_data = pd.DataFrame(index=sample_ids)
                
                for i, char in enumerate(sample_chars):
                    char_values = char.strip().split('\t')[1:]
                    char_values = [c.replace('"', '') for c in char_values]
                    if len(char_values) == len(sample_ids):
                        clinical_data[i] = char_values
    
    # Extract clinical features
    selected_clinical = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the selected clinical features
    preview = preview_df(selected_clinical)
    print("Preview of selected clinical features:")
    for key, value in preview.items():
        print(f"{key}: {value}")
    
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    
    # Save the clinical data
    selected_clinical.to_csv(out_clinical_data_file, index=True)
    print(f"Clinical data saved to {out_clinical_data_file}")


Preview of selected clinical features:
GSM899034: [0.0, 1.0]
GSM899035: [0.0, 1.0]
GSM899036: [0.0, 1.0]
GSM899037: [0.0, 1.0]
GSM899038: [0.0, 0.0]
GSM899039: [0.0, 0.0]
GSM899040: [0.0, 0.0]
GSM899041: [0.0, 1.0]
GSM899042: [0.0, 1.0]
GSM899043: [0.0, 1.0]
GSM899044: [0.0, 1.0]
GSM899045: [0.0, 0.0]
GSM899046: [0.0, 0.0]
GSM899047: [0.0, 0.0]
GSM899048: [0.0, 0.0]
GSM899049: [0.0, 1.0]
GSM899050: [0.0, 1.0]
GSM899051: [0.0, 0.0]
GSM899052: [0.0, 0.0]
GSM899053: [0.0, 0.0]
GSM899054: [0.0, 0.0]
GSM899055: [0.0, 0.0]
GSM899056: [0.0, 0.0]
GSM899057: [0.0, 1.0]
GSM899058: [0.0, 1.0]
GSM899059: [0.0, 1.0]
GSM899060: [0.0, 1.0]
GSM899061: [0.0, 0.0]
GSM899062: [0.0, 0.0]
GSM899063: [0.0, 0.0]
GSM899064: [0.0, 0.0]
GSM899065: [0.0, 0.0]
GSM899066: [0.0, 0.0]
GSM899067: [0.0, 0.0]
GSM899068: [0.0, 1.0]
GSM899069: [0.0, 1.0]
GSM899070: [0.0, 0.0]
GSM899071: [0.0, 0.0]
GSM899072: [0.0, 0.0]
GSM899073: [0.0, 0.0]
GSM899074: [1.0, 0.0]
GSM899075: [1.0, 0.0]
GSM899076: [1.0, 1.0]
GSM899077: [1.0

### Step 3: Gene Data Extraction

In [4]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. First, let's examine the structure of the matrix file to understand its format
import gzip

# Peek at the first few lines of the file to understand its structure
with gzip.open(matrix_file, 'rt') as file:
    # Read first 100 lines to find the header structure
    for i, line in enumerate(file):
        if '!series_matrix_table_begin' in line:
            print(f"Found data marker at line {i}")
            # Read the next line which should be the header
            header_line = next(file)
            print(f"Header line: {header_line.strip()}")
            # And the first data line
            first_data_line = next(file)
            print(f"First data line: {first_data_line.strip()}")
            break
        if i > 100:  # Limit search to first 100 lines
            print("Matrix table marker not found in first 100 lines")
            break

# 3. Now try to get the genetic data with better error handling
try:
    gene_data = get_genetic_data(matrix_file)
    print(gene_data.index[:20])
except KeyError as e:
    print(f"KeyError: {e}")
    
    # Alternative approach: manually extract the data
    print("\nTrying alternative approach to read the gene data:")
    with gzip.open(matrix_file, 'rt') as file:
        # Find the start of the data
        for line in file:
            if '!series_matrix_table_begin' in line:
                break
                
        # Read the headers and data
        import pandas as pd
        df = pd.read_csv(file, sep='\t', index_col=0)
        print(f"Column names: {df.columns[:5]}")
        print(f"First 20 row IDs: {df.index[:20]}")
        gene_data = df


Found data marker at line 72
Header line: "ID_REF"	"GSM899034"	"GSM899035"	"GSM899036"	"GSM899037"	"GSM899038"	"GSM899039"	"GSM899040"	"GSM899041"	"GSM899042"	"GSM899043"	"GSM899044"	"GSM899045"	"GSM899046"	"GSM899047"	"GSM899048"	"GSM899049"	"GSM899050"	"GSM899051"	"GSM899052"	"GSM899053"	"GSM899054"	"GSM899055"	"GSM899056"	"GSM899057"	"GSM899058"	"GSM899059"	"GSM899060"	"GSM899061"	"GSM899062"	"GSM899063"	"GSM899064"	"GSM899065"	"GSM899066"	"GSM899067"	"GSM899068"	"GSM899069"	"GSM899070"	"GSM899071"	"GSM899072"	"GSM899073"	"GSM899074"	"GSM899075"	"GSM899076"	"GSM899077"	"GSM899078"	"GSM899079"	"GSM899080"	"GSM899081"	"GSM899082"	"GSM899083"	"GSM899084"	"GSM899085"	"GSM899086"	"GSM899087"	"GSM899088"	"GSM899089"	"GSM899090"	"GSM899091"	"GSM899092"	"GSM899093"	"GSM899094"	"GSM899095"	"GSM899096"	"GSM899097"	"GSM899098"	"GSM899099"	"GSM899100"	"GSM899101"	"GSM899102"	"GSM899103"	"GSM899104"	"GSM899105"	"GSM899106"	"GSM899107"	"GSM899108"	"GSM899109"	"GSM899110"	"GSM899111"	"GSM899112"	"

Index(['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at', '1294_at',
       '1316_at', '1320_at', '1405_i_at', '1431_at', '1438_at', '1487_at',
       '1494_f_at', '1552256_a_at', '1552257_a_at', '1552258_at', '1552261_at',
       '1552263_at', '1552264_a_at', '1552266_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Looking at the gene identifiers in the expression data
# The identifiers like "1007_s_at", "1053_at", "117_at", etc. appear to be
# Affymetrix probe IDs from a microarray platform, not standard human gene symbols.
# These need to be mapped to official gene symbols for meaningful analysis.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database'], 'Target Description': ['U48705 /FEATURE=mRNA /DEFINITION=HSU48705 Human receptor tyrosine kinase DDR gene, complete cds', 'M87338 /FEATURE= /DEFINITION=HUMA1SBU Human replication factor C, 40-kDa subunit (A1) mRNA, complete cds', "X51757 /FEATURE=cds /DEFINITION=HSP70B Human heat-shock protein HSP70B' gene", 'X69699 /FEATURE= /DEF

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Looking at the dataframe preview, 'ID' column contains probe IDs, and 'Gene Symbol' contains gene symbols
prob_col = 'ID'
gene_col = 'Gene Symbol'

# 2. Get the gene mapping dataframe
mapping_df = get_gene_mapping(gene_annotation, prob_col, gene_col)

# 3. Apply the gene mapping to convert from probe-level to gene-level expression
gene_data = apply_gene_mapping(gene_data, mapping_df)

# Verify the type of identifiers in the gene_data
print("First 10 mapped gene symbols:")
print(gene_data.index[:10])

# Save the gene data to a CSV file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


First 10 mapped gene symbols:
Index(['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2M-AS1', 'A2ML1', 'A2MP1',
       'A4GALT', 'A4GNT', 'AA06'],
      dtype='object', name='Gene')


Gene expression data saved to ../../output/preprocess/Irritable_bowel_syndrome_(IBS)/gene_data/GSE36701.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Load the normalized gene data 
gene_data = pd.read_csv(out_gene_data_file, index_col=0)

# 2. Re-extract clinical features from the SOFT file to get proper clinical data
# Use the actual clinical data from the matrix file properly
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
background_info, clinical_data = get_background_and_clinical_data(matrix_file)

# 3. Create a correct clinical features dataframe
# First inspect what's in the clinical data
clinical_data_dict = get_unique_values_by_row(clinical_data)
print("Clinical data structure:")
print(clinical_data_dict)

# Based on the sample characteristics dictionary shown previously, 
# extract and process clinical features
selected_clinical_df = pd.DataFrame()

# Process disease state row manually to ensure correct mapping
disease_row = clinical_data.iloc[trait_row]
samples = [col for col in disease_row.index if col != "!Sample_geo_accession"]
trait_values = []

for sample in samples:
    value = disease_row[sample]
    if pd.isna(value):
        trait_values.append(None)
    else:
        if ":" in value:
            value = value.split(":", 1)[1].strip()
        
        if "IBS" in value:
            trait_values.append(1)  # IBS is our target trait
        elif "IBD" in value:
            trait_values.append(0)  # IBD is the control
        else:
            trait_values.append(None)

# Create dataframe with processed values
selected_clinical_df[trait] = trait_values
selected_clinical_df.index = samples

# Save the corrected clinical data
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
selected_clinical_df.to_csv(out_clinical_data_file)
print(f"Corrected clinical data saved to {out_clinical_data_file}")

# 4. Link the clinical and genetic data
linked_data = pd.DataFrame()
# Transpose gene data to have samples as rows and genes as columns
gene_data_t = gene_data.T
# Verify alignment of sample IDs between clinical and gene data
common_samples = list(set(selected_clinical_df.index) & set(gene_data_t.index))
if common_samples:
    gene_data_filtered = gene_data_t.loc[common_samples]
    clinical_data_filtered = selected_clinical_df.loc[common_samples]
    # Join the data
    linked_data = pd.concat([clinical_data_filtered, gene_data_filtered], axis=1)
    print(f"Linked data shape: {linked_data.shape}")
else:
    # Alternative linking approach if sample IDs don't directly match
    print("No common sample IDs found. Attempting alternative linking...")
    # The GSM ids in gene data columns may correspond to the sample IDs
    clinical_data_reset = selected_clinical_df.reset_index()
    clinical_data_reset.columns = ["Sample"] + list(clinical_data_reset.columns[1:])
    gene_data_cols = list(gene_data.columns)
    
    # Create merged dataframe
    data_dict = {trait: []}
    # Add trait values
    for col in gene_data_cols:
        sample_idx = clinical_data_reset.index[clinical_data_reset["Sample"] == col] if "Sample" in clinical_data_reset.columns else []
        if len(sample_idx) > 0:
            data_dict[trait].append(clinical_data_reset.loc[sample_idx[0], trait])
        else:
            data_dict[trait].append(None)
    
    # Add gene expression values
    for gene in gene_data.index:
        data_dict[gene] = list(gene_data.loc[gene])
    
    linked_data = pd.DataFrame(data_dict, index=gene_data_cols)
    print(f"Alternative linked data shape: {linked_data.shape}")

# 5. Handle missing values in the linked data
linked_data = handle_missing_values(linked_data, trait)
print(f"Data after handling missing values: {linked_data.shape}")

# 6. Determine whether the trait and some demographic features are severely biased
is_trait_biased, unbiased_linked_data = judge_and_remove_biased_features(linked_data, trait)

# 7. Conduct quality check and save the cohort information
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True, 
    is_trait_available=True, 
    is_biased=is_trait_biased, 
    df=unbiased_linked_data,
    note="Dataset contains gene expression data from patients with IBS and IBD, examining effects of relaxation response mind-body intervention."
)

# 8. If the linked data is usable, save it as a CSV file
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    unbiased_linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Data was determined to be unusable and was not saved")

Clinical data structure:
{0: ['tissue: Rectal Colon Biopsy'], 1: ['gender: M', 'gender: F'], 2: ['subject identifier: SD52930', 'subject identifier: SD52931', 'subject identifier: SD52932', 'subject identifier: SD52933', 'subject identifier: SD52934', 'subject identifier: SD52935', 'subject identifier: SD52936', 'subject identifier: SD52937', 'subject identifier: SD52938', 'subject identifier: SD52939', 'subject identifier: SD52940', 'subject identifier: SD52941', 'subject identifier: SD52942', 'subject identifier: SD52943', 'subject identifier: SD52944', 'subject identifier: SD52945', 'subject identifier: SD52946', 'subject identifier: SD52947', 'subject identifier: SD52948', 'subject identifier: SD52949', 'subject identifier: SD52950', 'subject identifier: SD52951', 'subject identifier: SD52952', 'subject identifier: SD52953', 'subject identifier: SD52954', 'subject identifier: SD52958', 'subject identifier: SD52959', 'subject identifier: SD52960', 'subject identifier: SD52961', 'sub

Data after handling missing values: (144, 21279)
For the feature 'Irritable_bowel_syndrome_(IBS)', the least common label is '0.0' with 28 occurrences. This represents 19.44% of the dataset.
The distribution of the feature 'Irritable_bowel_syndrome_(IBS)' in this dataset is fine.



Linked data saved to ../../output/preprocess/Irritable_bowel_syndrome_(IBS)/GSE36701.csv
